In [103]:
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import shutil
import os
import time
import sys
sys.path.append("../Revision_Exp2-QF-FB")
from pathlib import Path
import functools
print = functools.partial(print, flush=True)
import numpy as np 


from qiskit_library import *
import torch
import numpy as np
from random import randrange
import qiskit as qk
from qiskit import Aer
from qiskit import execute
import math
import sys
import random
import time
from tqdm import tqdm

# from qiskit import IBMQ
# IBMQ.delete_accounts()
# IBMQ.save_account('62d0e14364f490e45b5b5e0f6eebdbc083270ffffb660c7054219b15c7ce99ab4aa3b321309c0a9d0c3fc20086baece1376297dcdb67c7b715f9de1e4fa79efb')
# IBMQ.load_account()



interest_num = [3,6]
img_size = 4
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 2
inference_batch_size = 1


img_size = 16
layers = [8, 5]
resume_path="qfnet_p_best_models/mnist_01369_star_0.940916*16"

print("="*100)
print("Demo 3 on MNIST. This script is for batch of data generation.")
print("\tStart at:",time.strftime("%m/%d/%Y %H:%M:%S"))
print("\tProblems and issues, please contact Dr. Weiwen Jiang (wjiang2@nd.edu)")
print("\tEnjoy and Good Luck!")
print("="*100)
print()

Demo 3 on MNIST. This script is for batch of data generation.
	Start at: 09/04/2020 01:07:42
	Problems and issues, please contact Dr. Weiwen Jiang (wjiang2@nd.edu)
	Enjoy and Good Luck!



In [104]:
from lib_qc import * 
from lib_util import *
from lib_net import *

# Network Architecture: 2 layers and each layer contains 2 neurons


device = torch.device("cpu")


model = Net(img_size,layers,True,[[1,1,1,1],[1,1]],True,False,False,False,True).to(device)


print("=> loading checkpoint from '{}'<=".format(resume_path))
checkpoint = torch.load(resume_path, map_location=device)
epoch_init, acc = checkpoint["epoch"], checkpoint["acc"]
model.load_state_dict(checkpoint["state_dict"])


# for name, para in model.named_parameters():
#     if "fc" in name:
#         print(name,binarize(para))
#     else:
#         print(name, para)



=> loading checkpoint from 'qfnet_p_best_models/mnist_01369_star_0.9409'<=


<All keys matched successfully>

In [105]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.extensions import XGate, UnitaryGate
import qiskit
from math import sqrt 
import math
import copy

def get_index_list(input,target):
    index_list = []
    try:
        beg_pos = 0
        while True:
            find_pos = input.index(target,beg_pos)
            index_list.append(find_pos)
            beg_pos = find_pos+1
    except Exception as exception:        
        pass    
    return index_list

def change_sign(sign,bin):
    affect_num = [bin]
    one_positions = []
    
    try:
        beg_pos = 0
        while True:
            find_pos = bin.index("1",beg_pos)
            one_positions.append(find_pos)
            beg_pos = find_pos+1
    except Exception as exception:
        # print("Not Found")
        pass
    
    for k,v in sign.items():
        change = True
        for pos in one_positions:
            if k[pos]=="0":                
                change = False
                break
        if change:
            sign[k] = -1*v
    

def find_start(affect_count_table,target_num):
    for k in list(affect_count_table.keys())[::-1]:
        if target_num<=k:
            return k


def recursive_change(direction,start_point,target_num,sign,affect_count_table,quantum_gates):
    
    if start_point == target_num:
        # print("recursive_change: STOP")
        return
    
    gap = int(math.fabs(start_point-target_num))    
    step = find_start(affect_count_table,gap)
    change_sign(sign,affect_count_table[step])
    quantum_gates.append(affect_count_table[step])
    
    if direction=="r": 
        # print("recursive_change: From",start_point,"Right(-):",step)
        start_point = start_point - step
        direction = "l"
        recursive_change(direction,start_point,target_num,sign,affect_count_table,quantum_gates)
        
    else:        
        # print("recursive_change: From",start_point,"Left(+):",step)
        start_point = start_point + step
        direction = "r"
        recursive_change(direction,start_point,target_num,sign,affect_count_table,quantum_gates)
        
    

def guarntee_upper_bound_algorithm(sign,target_num,total_len,digits):        
    flag = "0"+str(digits)+"b"
    pre_num = 0
    affect_count_table = {}
    quantum_gates = []
    for i in range(digits):
        cur_num = pre_num + pow(2,i)
        pre_num = cur_num
        binstr_cur_num = format(cur_num,flag) 
        affect_count_table[int(pow(2,binstr_cur_num.count("0")))] = binstr_cur_num   
        
    if target_num in affect_count_table.keys():
        quantum_gates.append(affect_count_table[target_num])
        change_sign(sign,affect_count_table[target_num])    
    else:
        direction = "r"
        start_point = find_start(affect_count_table,target_num)
        quantum_gates.append(affect_count_table[start_point])
        change_sign(sign,affect_count_table[start_point])
        recursive_change(direction,start_point,target_num,sign,affect_count_table,quantum_gates)
    
    return quantum_gates

def qf_map_extract_from_weight(weights):    
    # Find Z control gates according to weights
    w = (weights.detach().cpu().numpy())
    total_len = len(w)
    target_num = np.count_nonzero(w == -1)
    if target_num > total_len/2:
        w = w*-1
    target_num = np.count_nonzero(w == -1)    
    digits = int(math.log(total_len,2))
    flag = "0"+str(digits)+"b"
    max_num = int(math.pow(2,digits))
    sign = {}
    for i in range(max_num):        
        sign[format(i,flag)] = +1
    quantum_gates = guarntee_upper_bound_algorithm(sign,target_num,total_len,digits)
    
    # for k,v in sign.items():
    #     print(k,v)
    # print(w)
        
    # Build the mapping from weight to final negative num 
    fin_sign = list(sign.values())
    fin_weig = [int(x) for x in list(w)]
    
    # print(fin_sign)
    # print(fin_weig)
    sign_neg_index = []    
    try:
        beg_pos = 0
        while True:
            find_pos = fin_sign.index(-1,beg_pos)            
            # qiskit_position = int(format(find_pos,flag)[::-1],2)                            
            sign_neg_index.append(find_pos)
            beg_pos = find_pos+1
    except Exception as exception:        
        pass    
    

    weight_neg_index = []
    try:
        beg_pos = 0
        while True:
            find_pos = fin_weig.index(-1,beg_pos)
            weight_neg_index.append(find_pos)
            beg_pos = find_pos+1
    except Exception as exception:        
        pass    
    map = {}
    for i in range(len(sign_neg_index)):
        map[sign_neg_index[i]] = weight_neg_index[i]
            
    # print(fin_sign)
    # print(fin_weig)
    # print(map)
    
    ret_index = list([-1 for i in range(len(fin_weig))])
    
    
    for k,v in map.items():
        ret_index[k]=v
    
    
    for i in range(len(fin_weig)):
        if ret_index[i]!=-1:
            continue
        for j in range(len(fin_weig)):
            if j not in ret_index:
                ret_index[i]=j
                break
    # print(ret_index)
    # sys.exit(0)     
    # for k,v in map.items():
    #     tmp1 = ret_index[k]
    #     tmp2 = ret_index[v]
    #     ret_index[k] = tmp2
    #     ret_index[v] = tmp1
    # 

    return quantum_gates,ret_index
    
    
def extract_model(model):
    layer_prop = {}
    batch_adj_prop = {}
    indiv_adj_prop = {}
    for name, para in model.named_parameters():
        if "fc" in name:
            layer_id = int(name.split(".")[0].split("c")[1])
            layer_prop[layer_id] = [para.shape[1],para.shape[0],binarize(para)]            
        elif "qca" in name:            
            if "l_0_5" in name or "running_rot" in name:
                layer_id = int(name.split(".")[0].split("a")[1])
                layer_fun = name.split(".")[1]
                if layer_id not in batch_adj_prop.keys():
                    batch_adj_prop[layer_id] = {}
                batch_adj_prop[layer_id][layer_fun] = para
        else:            
            print(name, para)
    
    # print(layer_prop)
    # print(batch_adj_prop)

    # First layer
    first_layer_num = layer_prop[0][1]
    
    first_layer_input_q = int(math.log(layer_prop[0][0],2))
    first_layer_addition_q = max(first_layer_input_q-2,0)        
    first_layer_batch_q = 0    
    if 0 in batch_adj_prop.keys():
        first_layer_batch_q = 2
    
    first_layer_q = first_layer_num*(first_layer_input_q+first_layer_batch_q)
    
    # print(first_layer_q)
    
    # Second layer    
    second_layer_num = layer_prop[1][1]
    if layer_prop[1][0]==0:
        second_layer_input_q=int(math.log(layer_prop[1][0],2))
        second_layer_encode_q = 0
        second_layer_output_q = 0
    else:
        second_layer_input_q = layer_prop[1][0]
        second_layer_encode_q = int(math.log(second_layer_input_q,2))    
        second_layer_output_q = int(math.log(second_layer_encode_q,2))
    
    second_layer_addition_q = max(first_layer_input_q-2,0)        
    
    second_layer_batch_q = 0
    if 1 in batch_adj_prop.keys():
        second_layer_batch_q = 2

    second_layer_q = second_layer_num*(second_layer_input_q+second_layer_output_q+second_layer_batch_q)
    # print(second_layer_q)
    sec_list = [second_layer_input_q, second_layer_num*second_layer_output_q,
                second_layer_num*second_layer_encode_q, second_layer_num*second_layer_batch_q]
    return first_layer_q,sec_list,first_layer_input_q,layer_prop,batch_adj_prop,max(first_layer_addition_q,second_layer_addition_q)
    
# Main part




def q_map_neural_compute_body(circ,inputs,iq,inference_batch_size,log_batch_size,weights):
    
    quantum_gates,ret_index = qf_map_extract_from_weight(weights)
    # print(ret_index)
    # print(quantum_gates)
    # 
    expand_for_batch_index = copy.deepcopy(ret_index)
    for b in range(inference_batch_size-1):
        start = len(ret_index)*(b+1)
        new_batch_index = [x+start for x in ret_index]
        expand_for_batch_index += new_batch_index
    index = torch.LongTensor(expand_for_batch_index)    
    Input0 = copy.deepcopy(Q_InputMatrix)
    Input0 = Input0[index]
    # print("for debug comparison")
    # print(Q_InputMatrix[:,0])
    # print(index)
    
    
    # print(Input0[:,0])
    circ.append(UnitaryGate(Input0, label="Input0"), inputs[0:iq])
    
    
    
    # print(circ)
    # backend = Aer.get_backend('unitary_simulator')
    # job = execute(circ, backend)
    # result = job.result()
    # torch.set_printoptions(threshold=sys.maxsize)
    # np.set_printoptions(threshold=sys.maxsize)
    # state = result.get_unitary(circ, decimals=4)
    # print(state[:,0])
    # 
    
    qbits = inputs[0:iq-log_batch_size]
    for gate in quantum_gates:
        z_count = gate.count("1")
        # z_pos = get_index_list(gate,"1")
        z_pos = get_index_list(gate[::-1],"1")
        # print(z_pos)
        if z_count==1:
            circ.z(qbits[z_pos[0]])
        elif z_count==2:
            circ.cz(qbits[z_pos[0]],qbits[z_pos[1]])
        elif z_count==3:
            qiskit_library.ccz(circ,qbits[z_pos[0]],qbits[z_pos[1]],qbits[z_pos[2]],aux_qr[0])
        elif z_count==4:
            qiskit_library.cccz(circ,qbits[z_pos[0]],qbits[z_pos[1]],qbits[z_pos[2]],qbits[z_pos[3]],aux_qr[0],aux_qr[1])
    
    # 
    # 
    # print(circ)
    # backend = Aer.get_backend('unitary_simulator')
    # job = execute(circ, backend)
    # result = job.result()
    # 
    # torch.set_printoptions(threshold=sys.maxsize)
    # np.set_printoptions(threshold=sys.maxsize)
    # state = result.get_unitary(circ, decimals=4)
    # print(state[:,0])
    # sys.exit(0)


def q_map_neural_compute_extract(circ,inputs,iq,outputs,log_batch_size):        
    qbits = inputs[0:iq-log_batch_size]
    # qbits = inputs[log_batch_size:iq]
    for q in qbits:
        circ.h(q)
    
    circ.barrier()
    
    for q in qbits:
        circ.x(q)
    
    
    digits = log_batch_size
    flag = "0"+str(digits)+"b"
    # qbits = inputs[0:log_batch_size]
    qbits = inputs[iq-log_batch_size:iq]
    if log_batch_size!=0:
        for i in range(int(math.pow(2,log_batch_size))):        
            binstr_cur_num = format(i,flag)[::-1]
            
            for pos in range(len(binstr_cur_num)):            
                if binstr_cur_num[pos]=="0":                
                    circ.x(qbits[pos])
                    
            if digits==1:                            
                qiskit_library.cccccx(circ, inputs[0:iq], outputs[i],aux_qr)            
            elif digits==2:
                qiskit_library.ccccccx(circ, inputs[0:iq], outputs[i],aux_qr)
            elif digits==3:
                qiskit_library.cccccccx(circ, inputs[0:iq], outputs[i],aux_qr)
            
            
            for pos in range(len(binstr_cur_num)):                
                if binstr_cur_num[pos]=="0":                
                    circ.x(qbits[pos])
            circ.barrier()
            
    else:
        qiskit_library.ccccx(circ, inputs[0], inputs[1], inputs[2], inputs[3], outputs[0], aux_qr[0], aux_qr[1])
    
    circ.barrier()
    qbits = inputs[log_batch_size:iq]
    for q in qbits:    
        circ.x(q)
    

def q_map_q_ori_net(circ, s_qr_in, s_qr_enc, aux_qr, weights):
    if len(weights)==4:
        for i in range(len(weights)):
            if weights[i]==-1:
                circ.x(s_qr_in[i])
        for i in range(2):
            circ.h(s_qr_enc[i])
        circ.barrier()
        circ.x(s_qr_enc[0])
        circ.x(s_qr_enc[1])
        ccz(circ,s_qr_in[3],s_qr_enc[0],s_qr_enc[1],aux_qr[0])
        circ.x(s_qr_enc[0])
        circ.x(s_qr_enc[1])
                
        circ.x(s_qr_enc[1])
        ccz(circ,s_qr_in[2],s_qr_enc[0],s_qr_enc[1],aux_qr[0])
        circ.x(s_qr_enc[1])
        
        circ.x(s_qr_enc[0])
        ccz(circ,s_qr_in[1],s_qr_enc[0],s_qr_enc[1],aux_qr[0])
        circ.x(s_qr_enc[0])
        
        
        ccz(circ,s_qr_in[0],s_qr_enc[0],s_qr_enc[1],aux_qr[0])
        
        circ.barrier()
        for i in range(len(weights)):
            if weights[i]==-1:
                circ.x(s_qr_in[i])
                
    elif len(weights)==8:
        print("Size", len(weights),"in 2nd layer is now not supportted")
        sys.exit(0)
    else:
        print("Size", len(weights),"in 2nd layer is now not supportted")
        sys.exit(0)


def q_map_bn(circ, s_qr_enc, s_qr_bat, output, aux_qr, enc_len, type, val):
    if enc_len!=2:
        print("Encoder size of ", enc_len,"is now not supportted")
        sys.exit(0)
    else:        
        for i in range(enc_len):
            circ.h(s_qr_enc[i])
            circ.x(s_qr_enc[i])
        circ.barrier()
        circ.ccx(s_qr_enc[0],s_qr_enc[1],s_qr_bat[0])        
        qc_ang = 2*torch.tensor(math.sqrt(val)).asin().item()        
        circ.ry(qc_ang,s_qr_bat[1])
        if type==1:
            circ.ccx(s_qr_bat[0],s_qr_bat[1],output)
            

def AinB(A,B):
    idx_a = []
    for i in range(len(A)):
        if A[i]=="1":
            idx_a.append(i)    
    flag = True
    for j in idx_a:
        if B[j]=="0":
            flag=False
            break
    return flag
            
        
def FFNN_Create_Weight(weights):        
    # Find Z control gates according to weights
    w = (weights.detach().cpu().numpy())
    total_len = len(w)            
    digits = int(math.log(total_len,2))
    flag = "0"+str(digits)+"b"
    max_num = int(math.pow(2,digits))
    sign = {}
    for i in range(max_num):        
        sign[format(i,flag)] = +1    
    sign_expect = {}
    for i in range(max_num):
        sign_expect[format(i,flag)] = int(w[i])    
    
    order_list = []
    for i in range(digits+1):
        for key in sign.keys():
            if key.count("1") == i:
                order_list.append(key)    
    
    gates = []    
    sign_cur = copy.deepcopy(sign_expect)
    for idx in range(len(order_list)):
        key = order_list[idx]
        if sign_cur[key] == -1:
            gates.append(key)
            for cor_idx in range(idx,len((order_list))):
                if AinB(key,order_list[cor_idx]):
                    sign_cur[order_list[cor_idx]] = (-1)*sign_cur[order_list[cor_idx]]    
    return gates


    

def gates_to_cost(gates):
    cost = 0
    for gate in gates:
        num_one = gate.count("1")
        if num_one==1:
            cost += 1
        else:
            cost += (num_one-1)*2-1
    return cost
        

def prob_extract_weights(weights):
    in_size = len(weights)
    out_qbit = int(math.log2(in_size))
    gates = []
    control_gates="1"
    
    # Encoding inputs
    for i in range(out_qbit):
        control_gates+="1"
    for i in range(in_size):
        gates.append(control_gates)
    
    # Encoding and Deconding weights
    for w in weights:
        if w==-1:
            gates.append("1")
            gates.append("1")
    
    # # H gates
    # for i in range(out_qbit):
    #     gates.append("1")
    # 
    gates.append(control_gates)
    
    return gates

import qiskit_library
f,sec_list,iq,nn_prop,bn_prop,aux = extract_model(model)
qf_net_cost = 0
ffnn_cost = 0

sec_layer_cost = 0

# print(nn_prop)
for weights in nn_prop[0][2]:        
    quantum_gates,ret_index = qf_map_extract_from_weight(weights)    
    qf_net_cost += gates_to_cost(quantum_gates)
    gates = FFNN_Create_Weight(weights)
    ffnn_cost += gates_to_cost(gates)
    
for weights in nn_prop[1][2]:
    gates = prob_extract_weights(weights)
    print(gates_to_cost(gates))
    sec_layer_cost += gates_to_cost(gates)
    
print("FFNN:", ffnn_cost)
print("QF-Net+:",qf_net_cost)
print("Second layer cost",sec_layer_cost)

51
59
47
47
47
FFNN: 5030
QF-Net+: 135
Second layer cost 251


In [106]:
print(int(math.log(img_size*img_size,2))*2-1)

15


In [107]:

sys.exit(0)

QF_Cost={}
FFNN_Cost={}
MLP_Cost={}

qf_net_cost = 0
ffnn_cost = 0
for e in range(4,12):
    QF_Cost[e]=[]
    FFNN_Cost[e]=[]
    MLP_Cost[e]=[]
    for r in range(50):
        MLP_Cost[e].append(2**e*2+e)
        
        x=torch.rand(2**e)-0.5       
        x = binarize(x)
        
        sq_cost = e+e*2-1
        
        quantum_gates,ret_index = qf_map_extract_from_weight(x)
        qf_net_cost = gates_to_cost(quantum_gates)+sq_cost
        
        gates = FFNN_Create_Weight(x)
        ffnn_cost = gates_to_cost(gates)+sq_cost
        
        QF_Cost[e].append(qf_net_cost)
        FFNN_Cost[e].append(ffnn_cost)
        
        print(e,2**e*2+e,qf_net_cost,ffnn_cost)

SystemExit: 0

/home/weiwen/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
for e in range(4,12):
    for i in range(50):    
        print(e,end=" ")

print()
for e in range(4,12):    
    for k in MLP_Cost[e]:
        print(k,end=" ")
print()
print()


for e in range(4,12):
    for i in range(50):    
        print(e,end=" ")
print()
for e in range(4,12):    
    for k in FFNN_Cost[e]:
        print(k,end=" ")
print()
print()


for e in range(4,12):
    for i in range(50):    
        print(e,end=" ")
print()
for e in range(4,12):    
    for k in QF_Cost[e]:
        print(k,end=" ")
        

In [ ]:
for e in range(4,12):        
    print(e,end=" ")

print()
for e in range(4,12):    
    print(float(sum(MLP_Cost[e])/len(MLP_Cost[e])),end=" ")
        
print()
print()


for e in range(4,12):    
    print(e,end=" ")
print()
for e in range(4,12):    
    print(float(sum(FFNN_Cost[e])/len(FFNN_Cost[e])),end=" ")
    
print()
print()


for e in range(4,12):    
    print(e,end=" ")
print()
for e in range(4,12):    
    print(float(sum(QF_Cost[e])/len(QF_Cost[e])),end=" ")
        
        
    